In [1]:
require 'torch'
require 'nn'
require 'rnn'
require 'csvigo'
require 'optim'
Plot = require 'itorch.Plot'

In [2]:
aroraname = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_shooting_first_sentence_numtext.csv'
wordfile = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/total_corpus_smry.csv'
nuggets = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/aurora_nuggets_numtext.csv'

In [99]:
dofile("utils.lua")

...Utils file loaded	



In [4]:
m = csvigo.load({path = aroraname, mode = "large"})
w = csvigo.load({path = wordfile, mode = "large"})
q = csvigo.load({path = nuggets, mode = "large"})

<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_shooting_first_sentence_numtext.csv	


<csv>	parsing done	
<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/total_corpus_smry.csv	


<csv>	parsing done	
<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/aurora_nuggets_numtext.csv	
<csv>	parsing done	


In [121]:
nuggs = grabNsamples(q, #q-1, nil)

139	elements read out of 	140	


In [5]:
N = 1000
K = 100
embed_dim = 6

In [6]:
--- Extracting N samples
out = grabNsamples(m, N, K)

mxl = 0
for k,v in pairs(out) do
    mxl = math.max(mxl, #v)
end

--- getting the length of the dictionary
vocab_size = 0
for k,v in pairs(out) do
    vocab_size = math.max(vocab_size, math.max(table.unpack(v)))
    if (k % N)==0 then
        print(k,'elements read out of ', #m)
        break
    end
end

--- Padding the data by the maximum length
xs = padZeros(out, mxl)

1000	elements read out of 	12510	


1000	elements read out of 	12510	


In [35]:
--- labels = torch.Tensor(torch.rand(#out)*20.):reshape(#out,1)
labels = torch.Tensor(torch.round(torch.rand(#out))):reshape(#out,1)

In [36]:
--- This is the correct format to input it
input = torch.LongTensor(xs)

In [37]:
LT = nn.LookupTableMaskZero(vocab_size, embed_dim)

-- For batch inputs, it's a little easier to start with sequence-length x batch-size tensor, so we transpose songData
myDataT = input:t()
batchLSTM = nn.Sequential()
batchLSTM:add(LT) -- will return a sequence-length x batch-size x embedDim tensor
batchLSTM:add(nn.SplitTable(1, embed_dim)) -- splits into a sequence-length table with batch-size x embedDim entries
-- print(batchLSTM:forward(myDataT)) -- sanity check
-- now let's add the LSTM stuff
batchLSTM:add(nn.Sequencer(nn.LSTM(embed_dim, embed_dim)))
batchLSTM:add(nn.SelectTable(-1)) -- selects last state of the LSTM
batchLSTM:add(nn.Linear(embed_dim, 1)) -- map last state to a score for classification
batchLSTM:add(nn.Linear(1, 1)) -- convert score to a probability
--- batchLSTM:add(nn.Sigmoid()) -- convert score to a probability
myPreds = batchLSTM:forward(myDataT)
print(#myPreds)
-- we can now call :backward() as follows
bceCrit = nn.MSECriterion()

loss = bceCrit:forward(myPreds, labels)
dLdPreds = bceCrit:backward(myPreds, labels)
batchLSTM:backward(myDataT, dLdPreds)
print(loss)

 1000
    1
[torch.LongStorage of size 2]



0.40899724714433	


In [38]:
inputs = input

In [39]:
target = labels

In [40]:
criterion = nn.MSECriterion()

In [41]:
epochs = 10
confusion = optim.ConfusionMatrix({0,1})

In [42]:
trainSet = {inputs}
trainSet.ivocab = vocab_size

In [43]:
lookupDim = 10

In [ ]:
for epoch=1, epochs do
   -- Single training epoch
   trainLoss = 0
   confusion:zero()
   model:training()
   for i, inputs, targets in myDataT:sampleiter(batchSize, epochSize) do
      xlua.progress(i, epochSize)
      trainInputs:resize(inputs:size()):copy(inputs)
      trainTargets:resize(targets:size()):copy(targets)

      local feval = function()
         gradParameters:zero()

         -- Forward
         local outputs = model:forward(trainInputs)
         local f = criterion:forward(outputs, trainTargets)
         trainLoss = trainLoss + f

         -- Backward
         local df_do = criterion:backward(outputs, trainTargets)
         model:backward(trainInputs, df_do)

         confusion:batchAdd(conOutputs, conTargets)
         return f, gradParameters
      end
      optimMethod(feval, parameters, optimState)
   end
   confusion:updateValids()
   if best_train_accu < confusion.totalValid then
      print("Best train accuracy: ".. best_train_accu ..
                  " current accu: ".. confusion.totalValid)
      best_train_accu = confusion.totalValid
      --best_train_model = model:clone()
   end
end

In [44]:
preds = {}
for i=1, myPreds:size()[1] do
    preds[i] = (myPreds[i][1] > 0.5) and 1 or 0 --- lua is stupid
end

In [50]:
ys = unpackZeros(preds)

In [131]:
#pred_summary

1000	


In [125]:
predsummary = buildPredSummary(preds, xs)

In [128]:
rougeRecall(predsummary, nuggs)

0.96040609137056	


In [129]:
rougePrecision(predsummary, nuggs)

0.087358020131129	


In [130]:
rougeF1(predsummary, nuggs)

0.16014897579143	


# Modeling MLP

In [170]:
pred_mlp = nn.Sequential()  -- A network that makes predictions given x.
pred_mlp:add(nn.Linear(5, 4))
pred_mlp:add(nn.Linear(4, 3))

xy_mlp = nn.ParallelTable() -- A network for predictions and for keeping the
xy_mlp:add(pred_mlp)        -- true label for comparison with a criterion
xy_mlp:add(nn.Identity())   -- by forwarding both x and y through the network.

mlp = nn.Sequential()       -- The main network that takes both x and y.
mlp:add(xy_mlp)             -- It feeds x and y to parallel networks;
cr = nn.MSECriterion()
cr_wrap = nn.CriterionTable(cr)
mlp:add(cr_wrap)            -- and then applies the criterion.

x = torch.randn(5)
y = torch.Tensor({1,1,1})
--- x = torch.randn(5)        -- Make input features.
--- y = torch.ones(5)      -- Popluates it with 3 elements near zero
loss = {}
for i = 1, 100 do           -- Do a few training iterations
   err = mlp:forward{x,y}   -- Forward both input and output.
   loss[i]= err               -- Print error from criterion.

   mlp:zeroGradParameters() -- Do backprop...
   mlp:backward({x, y})
   mlp:updateParameters(0.05)
end

In [171]:
-- scatter plots
plot = Plot():circle(torch.range(1, #loss), 
                loss, 'red', 'Loss Function'):draw()
plot:line(torch.range(1, #loss), 
                loss, 'red', ''):redraw()
plot:title('Scatter Plot Demo'):redraw()
plot:xaxis('Iterations'):yaxis('Negative Log-Likelihood'):redraw()
plot:legend(true)
plot:redraw()